In [26]:
import os

# Prepend the folder with espeak.exe to PATH
os.environ["PATH"] = r"C:\Program Files (x86)\eSpeak\command_line\command_line;" + os.environ["PATH"]


In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate

from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


## LJ Speech

In [28]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [29]:
import torch
print(torch.cuda.is_available())


False


In [30]:
"""net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()
"""
"""_ = utils.load_checkpoint("/path/to/pretrained_ljs.pth", net_g, None)"""

import sys
sys.path.append(r"C:\Users\MIKKY\vits-main\vits-main")

import torch
from models import SynthesizerTrn
import utils

device = "cpu"

# Initialize model
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model
)
net_g.to(device)
net_g.eval()

# Load checkpoint (CPU-only)
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_ljs.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
net_g.load_state_dict(checkpoint['model'])  # Use 'model' key

print("Model loaded successfully on CPU!")



C:\Users\MIKKY\AppData\Roaming\Python\Python38\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Model loaded successfully on CPU!


C:\Users\MIKKY\AppData\Local\Temp\ipykernel_6512\2145425827.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device

In [31]:
import os
os.system("espeak --version")


0

In [9]:
"""import os

# Prepend the folder that contains espeak.exe to PATH
os.environ["PATH"] = r"C:\Program Files (x86)\eSpeak\command_line\command_line;" + os.environ["PATH"]


"""

'import os\n\n# Prepend the folder that contains espeak.exe to PATH\nos.environ["PATH"] = r"C:\\Program Files (x86)\\eSpeak\\command_line\\command_line;" + os.environ["PATH"]\n\n\n'

In [32]:
net_g.to(device)
net_g.eval()
checkpoint = torch.load(checkpoint_path, map_location=device)
net_g.load_state_dict(checkpoint['model'])  # use 'model' key in checkpoint


C:\Users\MIKKY\AppData\Local\Temp\ipykernel_6512\1787058031.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)

<All keys matched successfully>

In [36]:
import os
import torch
from models import SynthesizerTrn
import utils
from text import text_to_sequence
from text.symbols import symbols   # ✅ import here
import scipy.io.wavfile as wavfile

# 1️⃣ Use CPU
device = "cpu"

# 2️⃣ Load configs
hps = utils.get_hparams_from_file("configs/ljs_base.json")

# 3️⃣ Initialize model
net_g = SynthesizerTrn(
    len(symbols),   # ✅ use imported symbols
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model
)
net_g.to(device)
net_g.eval()

# 4️⃣ Load pretrained checkpoint
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_ljs.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
net_g.load_state_dict(checkpoint['model'])

# 5️⃣ Convert text → IDs
text = "VITS is Awesome!"
stn_tst = torch.LongTensor(
    text_to_sequence(text, hps.data.text_cleaners)  # ✅ only 2 args
)


# 6️⃣ Prepare input
x_tst = stn_tst.unsqueeze(0).to(device)
x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)

# 7️⃣ Inference
with torch.no_grad():
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().numpy()

# 8️⃣ Save to wav
wavfile.write("output.wav", hps.data.sampling_rate, audio.astype('float32'))
print("✅ Done! Audio saved as output.wav")


C:\Users\MIKKY\AppData\Local\Temp\ipykernel_6512\2637349133.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device

✅ Done! Audio saved as output.wav


In [38]:
"""import os
import torch
from phonemizer.backend import EspeakBackend
from phonemizer import phonemize
from models import SynthesizerTrn
import utils

# 1️⃣ Force Python to see espeak
os.environ["PATH"] = r"C:\Program Files (x86)\eSpeak\command_line\command_line;" + os.environ["PATH"]

# 2️⃣ Use CPU
device = "cpu"

# 3️⃣ Initialize model (replace `symbols` and `hps` with your repo values)
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model
)
net_g.to(device)
net_g.eval()

# 4️⃣ Load checkpoint
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_ljs.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
net_g.load_state_dict(checkpoint['model'])

# 5️⃣ Initialize Espeak backend manually for Windows
es_backend = EspeakBackend('en-us')  # phonemizer will now find espeak.exe in PATH

# 6️⃣ Convert text to phonemes
text = "VITS is Awesome!"
stn_tst = phonemize(
    text,
    backend="espeak",     # ✅ use string, not object
    language="en-us",
    strip=True,
    preserve_punctuation=True
)

text = "VITS is Awesome!"
stn_tst = phonemize(
    text,
    backend=es_backend,
    language='en-us',
    strip=True,
    preserve_punctuation=True
)

# 7️⃣ Prepare input tensor
x_tst = torch.LongTensor([stn_tst]).unsqueeze(0).to(device)
x_tst_lengths = torch.LongTensor([len(stn_tst)]).to(device)

# 8️⃣ Infer audio
with torch.no_grad():
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().numpy()

# 9️⃣ Save to file
import scipy.io.wavfile as wavfile
wavfile.write("output.wav", hps.data.sampling_rate, audio.astype('float32'))

print("Done! Audio saved as output.wav")
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 685-686: truncated \UXXXXXXXX escape (2869042281.py, line 1)

In [39]:
from phonemizer import phonemize

text = "hello world"
print(phonemize(text, language='en-us', backend='espeak'))


həloʊ wɜːld 


In [41]:
"""stn_tst = get_text("VITS is Awesome!", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))"""
# ✅ CPU-safe inference

# 1️⃣ Convert text → sequence
stn_tst = get_text("VITS is Awesome!", hps)

# 2️⃣ Inference
with torch.no_grad():
    # send to CPU
    x_tst = stn_tst.unsqueeze(0).to(device)  
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)

    # run model
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().numpy()

# 3️⃣ Play or save audio
import IPython.display as ipd
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))


## VCTK

In [42]:
hps = utils.get_hparams_from_file("./configs/vctk_base.json")

In [46]:
device = torch.device("cpu")  # CPU-only

# Initialize model on CPU
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model
).to(device)
_ = net_g.eval()

# Load the pretrained VCTK checkpoint
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_vctk.pth"
_ = utils.load_checkpoint(checkpoint_path, net_g, None)


c:\Users\MIKKY\vits-main\vits-main\utils.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')


INFO:root:Loaded checkpoint 'C:\Users\MIKKY\vits-main\vits-main\pretrained_vctk.pth' (iteration 0)


In [48]:
# pick device
device = torch.device("cpu")

# convert text → tensor
stn_tst = get_text("VITS is Awesome!", hps)

with torch.no_grad():
    # send to CPU
    x_tst = stn_tst.unsqueeze(0).to(device)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    sid = torch.LongTensor([4]).to(device)  # target speaker ID

    # inference
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        sid=sid,
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().float().numpy()

# play audio
import IPython.display as ipd
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))


### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))